In [1]:
import os
import string
import datetime

from data import preproc as pp, evaluation
from data.generator import DataGenerator, Tokenizer
from network.model import HTRModel

In [2]:
input_size = (128, 64, 1)
max_text_length = 32
charset_base = string.printable[:95]

source='iam'
arch ="flor" # puigcerver, bluche, flor,
batch_size = 100
type_of_run = 'test'
if type_of_run == 'train':
    train_model = False
else:
    train_model = True

In [3]:
source_path = os.path.join("/floyd/input/words_htr_onew", f"{source}_tvt.hdf5")
output_path = os.path.join("/floyd/home/output_words", source, arch)
target_path = os.path.join(output_path, "checkpoint_weights.hdf5")

assert os.path.isfile(source_path) or os.path.isfile(target_path)

In [4]:
dtgen = DataGenerator(source=source_path,
                      batch_size=batch_size,
                      charset=charset_base,
                      max_text_length=max_text_length,
                      predict=train_model)

model = HTRModel(architecture=arch,
                 input_size=input_size,
                 vocab_size=dtgen.tokenizer.vocab_size,
                 beam_width=10,
                 stop_tolerance=20,
                 reduce_tolerance=15)

model.compile(learning_rate=0.001)
model.load_checkpoint(target=target_path)

model.summary(output_path, "summary.txt")
callbacks = model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

/floyd/input/words_htr_onew/iam_tvt.hdf5
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 32, 16)        160       
_________________________________________________________________
p_re_lu (PReLU)              (None, 64, 32, 16)        16        
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 32, 16)        112       
_________________________________________________________________
full_gated_conv2d (FullGated (None, 64, 32, 16)        4640      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 32, 32)        4640      
_________________________________________________________________
p_re_lu_1 (PReLU)   

In [5]:
print(f"Test images: {dtgen.size['test']}")

Test images: 6603


In [6]:
start_time = datetime.datetime.now()

predicts, _ = model.predict(x=dtgen.next_test_batch(),
                            steps=dtgen.steps['test'],
                            ctc_decode=True,
                            verbose=1)

predicts = [dtgen.tokenizer.decode(x[0]) for x in predicts]

total_time = datetime.datetime.now() - start_time

with open(os.path.join(output_path, "predict.txt"), "w") as lg:
    for pd, gt in zip(predicts, dtgen.dataset['test']['gt']):
        print("Ground Truth:", gt)
        print("Predicted:", pd)
        lg.write(f"TE_L {gt}\nTE_P {pd}\n")

evaluate = evaluation.ocr_metrics(predicts=predicts,
                                  ground_truth=dtgen.dataset['test']['gt'],
                                  norm_accentuation=norm_accentuation,
                                  norm_punctuation=norm_punctuation)

e_corpus = "\n".join([
    f"Total test images:    {dtgen.size['test']}",
    f"Total time:           {total_time}",
    f"Time per item:        {total_time / dtgen.size['test']}\n",
    f"Metrics:",
    f"Character Error Rate: {evaluate[0]:.8f}",
    f"Word Error Rate:      {evaluate[1]:.8f}",
    f"Sequence Error Rate:  {evaluate[2]:.8f}"
])

sufix = ("_norm" if norm_accentuation or norm_punctuation else "") + \
        ("_accentuation" if norm_accentuation else "") + \
        ("_punctuation" if norm_punctuation else "")

with open(os.path.join(output_path, f"evaluate{sufix}.txt"), "w") as lg:
    lg.write(e_corpus)
    print(e_corpus)

Model Predict
67/67 [==============================] - 2s 37ms/step
CTC Decode
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
67/67 [==============================] - 52s 782ms/step
Ground Truth: first
Predicted: first
Ground Truth: pots
Predicted: pots
Ground Truth: the
Predicted: the
Ground Truth: Had
Predicted: Aad
Ground Truth: such
Predicted: ae
Ground Truth: But
Predicted: But
Ground Truth: performance
Predicted: pelmn
Ground Truth: when
Predicted: when
Ground Truth: fly
Predicted: fae
Ground Truth: were
Predicted: we
Ground Truth: which
Predicted: which
Ground Truth: effects
Predicted: efeds
Ground Truth: severity
Predicted: set
Ground Truth: This
Predicted: Ths
Ground Truth: their
Predicted: thir
Ground Truth: produced
Predicted: prodced
Ground Truth: must
Predicted: mut
Ground Truth: Andrew
Predicted: Adren
Ground Truth: does
Predicted: doe
Ground Truth: time
Predicted: tie
Ground Truth: the
Predicted: the
Ground Truth: healt

NameError: name 'norm_accentuation' is not defined